In [1]:
# 2 weeks together
import pandas as pd
import numpy as np
import datetime
import os
# Add the new folders of Jian later

In [2]:
def recursive_file_gen(my_root_dir):
    for root, dirs, files in os.walk(my_root_dir):
        for file in files:
            yield os.path.join(root, file)
            
last_saturday=datetime.datetime.now().date()-datetime.timedelta(days=datetime.datetime.now().date().weekday()+2) #+7

write_folder="/home/jian/celery/BiWeekly_New_SignUps/monthly_update/output_"+str(last_saturday)+"/"
try:
    os.stat(write_folder)
except:
    os.mkdir(write_folder)

week_beginning=last_saturday-datetime.timedelta(days=14) 

recent_2_weeks=[last_saturday-datetime.timedelta(days=x*7) for x in range(2)]
recent_2_weeks

[datetime.date(2020, 6, 13), datetime.date(2020, 6, 6)]

In [3]:
# Checking for the recent weeks that haven't moved to the folder
possible_recent_folders=["/home/jian/BigLots/MediaStorm_"+str(x)+"/" for x in recent_2_weeks]
recent_file_list=[]
for dirc in possible_recent_folders:
    list_recent=[x for x in list(recursive_file_gen(dirc)) if ("Master" in x) & (".txt" in x) ]
    recent_file_list=recent_file_list+list_recent
recent_file_df=pd.DataFrame({"path":recent_file_list,"date":[datetime.datetime.strptime(x.split("MasterWeekly")[1][:8],"%Y%m%d").date()-datetime.timedelta(days=3) for x in recent_file_list]},index=[x for x in range(len(recent_file_list))])


list_1_after_201806=[x for x in list(recursive_file_gen("/home/jian/BigLots/2019_by_weeks/")) if ("Master" in x) & (".txt" in x) ]
list_2_after_201806=[x for x in list(recursive_file_gen("/home/jian/BigLots/2020_by_weeks/")) if ("Master" in x) & (".txt" in x) ]
list_1_after_201806=list_1_after_201806+list_2_after_201806
folder_date=[datetime.datetime.strptime(x.split("/")[len(x.split("/"))-2].split("_")[1],"%Y-%m-%d").date() for x in list_1_after_201806]
df_1_after_201806=pd.DataFrame({"date":folder_date,"path":list_1_after_201806},index=[x for x in range(len(list_1_after_201806))])
df_1_after_201806['date'].apply(lambda x: x.weekday()).unique()
df_1_after_201806=df_1_after_201806.sort_values("date").reset_index()
df_1_after_201806=df_1_after_201806[df_1_after_201806['date']>week_beginning]
del df_1_after_201806['index']

new_2_biweekly_files=df_1_after_201806.append(recent_file_df)

print(len(new_2_biweekly_files))
new_2_biweekly_files=new_2_biweekly_files['path'].unique().tolist()

2


In [4]:
new_2_biweekly_files

['/home/jian/BigLots/2020_by_weeks/MediaStorm_2020-06-06/MediaStormMasterWeekly20200609-115743-386.txt',
 '/home/jian/BigLots/MediaStorm_2020-06-13/MediaStormMasterWeekly20200616-113200-395.txt']

In [5]:
if len(new_2_biweekly_files)==2: # Changed to weekly
    df_all_new_master = pd.DataFrame()
    for file in new_2_biweekly_files:
        df = pd.read_csv(file,nrows = None,sep= '|',usecols = ['customer_id_hashed','email_address_hash','customer_zip_code'],
                         dtype =str)
        df_all_new_master = df_all_new_master.append(df,ignore_index = True)
        print(df_all_new_master.shape)
    print(len(df_all_new_master.index))
    df_all_new_master = df_all_new_master.drop_duplicates('email_address_hash')
    print(len(df_all_new_master.index))


(196064, 3)
(385589, 3)
385589
385584


In [6]:
# Remove previous week ids

previous_id_files_lists=list(recursive_file_gen("/home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2020Q1/final_segments_uploaded_LR_0_18/"))

# previous_id_files_lists=[x for x in previous_id_files_lists if "Copy of 48 Stores_727.csv" not in x]
previous_id_files_lists=[x for x in previous_id_files_lists if ".csv" in x]
print(len(previous_id_files_lists))

273


In [7]:
previous_id_files_lists

['/home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2020Q1/final_segments_uploaded_LR_0_18/T_Legacy_Active_H_P_Above50K_MultiFamilyOrCondo_CompetitorsHigh_FurinitureHigh_RFM20Q1.csv',
 '/home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2020Q1/final_segments_uploaded_LR_0_18/T_Legacy_Active_H_P_Above50K_MultiFamilyOrCondo_CompetitorsHigh_FurinitureLow_RFM20Q1.csv',
 '/home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2020Q1/final_segments_uploaded_LR_0_18/T_Legacy_Active_H_P_Above50K_MultiFamilyOrCondo_CompetitorsLow_FurinitureHigh_RFM20Q1.csv',
 '/home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2020Q1/final_segments_uploaded_LR_0_18/T_Legacy_Active_H_P_Above50K_MultiFamilyOrCondo_CompetitorsLow_FurinitureLow_RFM20Q1.csv',
 '/home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2020Q1/final_segments_uploaded_LR_0_18/T_Legacy_Active_H_P_Above50K_SingleFamily_CompetitorsHigh_FurinitureHigh_RFM20Q1.csv',
 '/home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_20

In [8]:
import glob
uploaded_new_seg=list(recursive_file_gen("/home/jian/celery/BiWeekly_New_SignUps/monthly_update/"))
uploaded_new_seg=[x for x in uploaded_new_seg if ".csv" in x]

uploaded_new_seg.sort()
uploaded_new_seg=uploaded_new_seg[-24:]

print("len(uploaded_new_seg)"+str(len(uploaded_new_seg)))
previous_id_files_lists=previous_id_files_lists+uploaded_new_seg

len(uploaded_new_seg)24


In [9]:
# Exclude the quarterly regular segments


In [10]:
for file_uploaded in previous_id_files_lists:
    df = pd.read_csv(file_uploaded,usecols = ['email_address_hash'])
    previous_email_set=set(df['email_address_hash'].tolist())
    df_all_new_master = df_all_new_master[~df_all_new_master['email_address_hash'].isin(previous_email_set)]
    print(datetime.datetime.now(),len(df_all_new_master.index),file_uploaded)

2020-06-17 10:11:54.865767 385584 /home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2020Q1/final_segments_uploaded_LR_0_18/T_Legacy_Active_H_P_Above50K_MultiFamilyOrCondo_CompetitorsHigh_FurinitureHigh_RFM20Q1.csv
2020-06-17 10:11:58.594399 385584 /home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2020Q1/final_segments_uploaded_LR_0_18/T_Legacy_Active_H_P_Above50K_MultiFamilyOrCondo_CompetitorsHigh_FurinitureLow_RFM20Q1.csv
2020-06-17 10:11:59.209790 385584 /home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2020Q1/final_segments_uploaded_LR_0_18/T_Legacy_Active_H_P_Above50K_MultiFamilyOrCondo_CompetitorsLow_FurinitureHigh_RFM20Q1.csv
2020-06-17 10:11:59.390995 385584 /home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2020Q1/final_segments_uploaded_LR_0_18/T_Legacy_Active_H_P_Above50K_MultiFamilyOrCondo_CompetitorsLow_FurinitureLow_RFM20Q1.csv
2020-06-17 10:12:31.533783 385584 /home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2020Q1/final_segments_uploaded_LR_

2020-06-17 10:14:56.315751 385565 /home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2020Q1/final_segments_uploaded_LR_0_18/T_Legacy_Active_H_S_Unknown_Unknown_CompetitorsLow_FurinitureHigh_RFM20Q1.csv
2020-06-17 10:14:56.576926 385565 /home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2020Q1/final_segments_uploaded_LR_0_18/T_Legacy_Active_H_S_Unknown_Unknown_CompetitorsLow_FurinitureLow_RFM20Q1.csv
2020-06-17 10:14:56.686183 385565 /home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2020Q1/final_segments_uploaded_LR_0_18/T_Legacy_Active_L_P_Above50K_MultiFamilyOrCondo_CompetitorsHigh_FurinitureHigh_RFM20Q1.csv
2020-06-17 10:14:56.821963 385565 /home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2020Q1/final_segments_uploaded_LR_0_18/T_Legacy_Active_L_P_Above50K_MultiFamilyOrCondo_CompetitorsHigh_FurinitureLow_RFM20Q1.csv
2020-06-17 10:14:56.928468 385565 /home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2020Q1/final_segments_uploaded_LR_0_18/T_Legacy_Active_L_P

2020-06-17 10:15:05.941866 385563 /home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2020Q1/final_segments_uploaded_LR_0_18/T_Legacy_Active_L_S_Unknown_Unknown_CompetitorsHigh_FurinitureLow_RFM20Q1.csv
2020-06-17 10:15:06.122389 385563 /home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2020Q1/final_segments_uploaded_LR_0_18/T_Legacy_Active_L_S_Unknown_Unknown_CompetitorsLow_FurinitureHigh_RFM20Q1.csv
2020-06-17 10:15:06.243906 385563 /home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2020Q1/final_segments_uploaded_LR_0_18/T_Legacy_Active_L_S_Unknown_Unknown_CompetitorsLow_FurinitureLow_RFM20Q1.csv
2020-06-17 10:15:06.422539 385563 /home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2020Q1/final_segments_uploaded_LR_0_18/T_Legacy_Active_M_P_Above50K_MultiFamilyOrCondo_CompetitorsHigh_FurinitureHigh_RFM20Q1.csv
2020-06-17 10:15:06.574046 385563 /home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2020Q1/final_segments_uploaded_LR_0_18/T_Legacy_Active_M_P_Above50K_Mu

2020-06-17 10:16:40.670854 385555 /home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2020Q1/final_segments_uploaded_LR_0_18/T_Legacy_Active_M_S_Unknown_Unknown_CompetitorsHigh_FurinitureHigh_RFM20Q1.csv
2020-06-17 10:16:41.193605 385554 /home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2020Q1/final_segments_uploaded_LR_0_18/T_Legacy_Active_M_S_Unknown_Unknown_CompetitorsHigh_FurinitureLow_RFM20Q1.csv
2020-06-17 10:16:47.584524 385554 /home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2020Q1/final_segments_uploaded_LR_0_18/T_Legacy_Active_M_S_Unknown_Unknown_CompetitorsLow_FurinitureHigh_RFM20Q1.csv
2020-06-17 10:16:48.074155 385554 /home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2020Q1/final_segments_uploaded_LR_0_18/T_Legacy_Active_M_S_Unknown_Unknown_CompetitorsLow_FurinitureLow_RFM20Q1.csv
2020-06-17 10:17:20.850137 385552 /home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2020Q1/final_segments_uploaded_LR_0_18/C_all_quarterly_segments_RFM20Q1.csv
2020-0

2020-06-17 10:18:15.021423 385549 /home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2020Q1/final_segments_uploaded_LR_0_18/T_SOTF_Active_H_P_Above50K_MultiFamilyOrCondo_CompetitorsHigh_FurinitureHigh_RFM20Q1.csv
2020-06-17 10:18:15.546106 385549 /home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2020Q1/final_segments_uploaded_LR_0_18/T_SOTF_Active_H_P_Above50K_MultiFamilyOrCondo_CompetitorsHigh_FurinitureLow_RFM20Q1.csv
2020-06-17 10:18:16.100589 385549 /home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2020Q1/final_segments_uploaded_LR_0_18/T_SOTF_Active_H_P_Above50K_MultiFamilyOrCondo_CompetitorsLow_FurinitureHigh_RFM20Q1.csv
2020-06-17 10:18:16.494668 385549 /home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2020Q1/final_segments_uploaded_LR_0_18/T_SOTF_Active_H_P_Above50K_MultiFamilyOrCondo_CompetitorsLow_FurinitureLow_RFM20Q1.csv
2020-06-17 10:18:42.208714 385541 /home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2020Q1/final_segments_uploaded_LR_0_18/T_S

2020-06-17 10:19:26.419598 385529 /home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2020Q1/final_segments_uploaded_LR_0_18/T_SOTF_Active_L_P_Above50K_SingleFamily_CompetitorsHigh_FurinitureHigh_RFM20Q1.csv
2020-06-17 10:19:26.670669 385529 /home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2020Q1/final_segments_uploaded_LR_0_18/T_SOTF_Active_L_P_Above50K_SingleFamily_CompetitorsHigh_FurinitureLow_RFM20Q1.csv
2020-06-17 10:19:27.080762 385529 /home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2020Q1/final_segments_uploaded_LR_0_18/T_SOTF_Active_L_P_Above50K_SingleFamily_CompetitorsLow_FurinitureHigh_RFM20Q1.csv
2020-06-17 10:19:27.204957 385529 /home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2020Q1/final_segments_uploaded_LR_0_18/T_SOTF_Active_L_P_Above50K_SingleFamily_CompetitorsLow_FurinitureLow_RFM20Q1.csv
2020-06-17 10:19:27.342896 385529 /home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2020Q1/final_segments_uploaded_LR_0_18/T_SOTF_Active_L_P_Below50K_

2020-06-17 10:20:12.397734 385527 /home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2020Q1/final_segments_uploaded_LR_0_18/T_SOTF_Active_M_P_Unknown_Unknown_CompetitorsHigh_FurinitureHigh_RFM20Q1.csv
2020-06-17 10:20:13.207757 385527 /home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2020Q1/final_segments_uploaded_LR_0_18/T_SOTF_Active_M_P_Unknown_Unknown_CompetitorsHigh_FurinitureLow_RFM20Q1.csv
2020-06-17 10:20:18.744999 385527 /home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2020Q1/final_segments_uploaded_LR_0_18/T_SOTF_Active_M_P_Unknown_Unknown_CompetitorsLow_FurinitureHigh_RFM20Q1.csv
2020-06-17 10:20:18.912239 385527 /home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2020Q1/final_segments_uploaded_LR_0_18/T_SOTF_Active_M_P_Unknown_Unknown_CompetitorsLow_FurinitureLow_RFM20Q1.csv
2020-06-17 10:20:19.305754 385527 /home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2020Q1/final_segments_uploaded_LR_0_18/T_SOTF_Active_M_S_Above50K_MultiFamilyOrCondo_Compe

2020-06-17 10:20:54.652936 385525 /home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2020Q1/final_segments_uploaded_LR_0_18/T_SOTF_Lapsed1318_L_S_Below50K_SingleFamily_CompetitorsLow_FurinitureHigh_RFM20Q1.csv
2020-06-17 10:20:57.970038 385525 /home/jian/celery/BiWeekly_New_SignUps/monthly_update/output_2020-02-22/NewReward_2020-02-22_P.csv
2020-06-17 10:20:58.598136 385525 /home/jian/celery/BiWeekly_New_SignUps/monthly_update/output_2020-02-22/NewReward_2020-02-22_S.csv
2020-06-17 10:21:01.284267 385525 /home/jian/celery/BiWeekly_New_SignUps/monthly_update/output_2020-02-22/NewReward_2020-02-22_T.csv
2020-06-17 10:21:04.258143 385524 /home/jian/celery/BiWeekly_New_SignUps/monthly_update/output_2020-03-07/NewReward_2020-03-07_P.csv
2020-06-17 10:21:04.497912 385524 /home/jian/celery/BiWeekly_New_SignUps/monthly_update/output_2020-03-07/NewReward_2020-03-07_S.csv
2020-06-17 10:21:07.733097 385524 /home/jian/celery/BiWeekly_New_SignUps/monthly_update/output_2020-03-07/NewReward_2020

In [11]:
df_all_new_master = df_all_new_master.drop_duplicates('email_address_hash')
df_all_new_master = df_all_new_master.drop_duplicates('customer_id_hashed')
print(len(df_all_new_master.index))

385518


In [12]:
zipmap = pd.ExcelFile('/home/jian/Projects/Big_Lots/New_TA/TA_created_in_201906/final_output_20190718/BL_final_TA_updated_JL_2019-07-18.xlsx')
zipmap=zipmap.parse("unique_zips_full_footprint",dtype=str)
zipmap['zip_type']=zipmap['zip_type'].replace("zips_10","trans_S")

zipmap['zipcodegroup'] = zipmap['zip_type'].apply(lambda x: x.split("_")[1])
print(zipmap.shape)
print(zipmap['zip_cd'].nunique())

zipmap=zipmap.rename(columns={"zip_cd":"customer_zip_code"})

zipmap=zipmap[['customer_zip_code','zipcodegroup']]
zipmap.head(2)

(12980, 3)
12980


,customer_zip_code,zipcodegroup
0,75040,P
1,20743,P


In [13]:
df_all_new_master = pd.merge(df_all_new_master,zipmap,on ='customer_zip_code',how = 'left' )
print(df_all_new_master['zipcodegroup'].unique())

df_all_new_master['zipcodegroup'].fillna('T',inplace = True)
print(df_all_new_master['zipcodegroup'].unique())

last_saturday_str="_"+str(last_saturday).split("-")[1]+str(last_saturday).split("-")[2]+str(last_saturday).split("-")[0][2:]+"_"

for revenue_flag in ['P','S','T']:
    df = df_all_new_master[df_all_new_master['zipcodegroup']==revenue_flag]
    df = df[['customer_id_hashed','email_address_hash','customer_zip_code']]
    df['segment_new_signups'] = 'NewReward' + last_saturday_str + revenue_flag
    print(revenue_flag,len(df.index))
    df.to_csv(write_folder+'NewReward_'+str(last_saturday)+'_'+revenue_flag+'.csv',index = False)
# To be uploaded to lr-big-lots /uploads/big_lots_onboarding

['P' 'S' nan]
['P' 'S' 'T']
P 120255
S 49216
T 216047


In [14]:
df_all_new_master.groupby(['zipcodegroup'])['email_address_hash'].count().to_frame().reset_index()

,zipcodegroup,email_address_hash
0,P,120255
1,S,49216
2,T,216047


In [15]:
df.head(2)

,customer_id_hashed,email_address_hash,customer_zip_code,segment_new_signups
2,369a46defee87aa39848c105aedf5799911aef336803d1...,7070bef27664509bb7a029ef207f4d76cd1fb747f53b23...,66048,NewReward_061320_T
3,83848d68b9c2cd4345848cdd511b4dbe94b6a84fef45dd...,4def195f57d97547f76766213ea5cf9ea34eb0a5a44dd8...,NaN,NewReward_061320_T


In [16]:
df_all_new_master.head(2)

,customer_id_hashed,email_address_hash,customer_zip_code,zipcodegroup
0,46bc45a71a18bce995652340af8a67f3a8e06f8544691c...,1ea1b7c2c58e8f98c4631b9435712b56e57ed428966972...,08330,P
1,6c1cfdbd34b2d776074bba00f6e36bc3aa30968f668868...,978f80e29f330fe64c98c601a406d2eb7f4175f96d3423...,17601,S


In [17]:
df_all_new_master[pd.isnull(df_all_new_master['customer_zip_code'])].shape

(196947, 4)

In [18]:
# to do below 
print(datetime.datetime.now())

2020-06-17 10:24:22.581223


In [19]:
import paramiko
import glob

host = "files.liveramp.com"
port = 22
password = "Biglots2018!"
username = "lr-big-lots"

transport = paramiko.Transport((host, port))

transport.connect(username = username, password = password)
sftp = paramiko.SFTPClient.from_transport(transport)

# local_path defined above before saving the local txt
local_list=glob.glob(write_folder+"*.csv")
local_list=[x for x in local_list if 'NewReward_'+str(last_saturday)+'_' in x]

for local_path in local_list:
    remote_path="/uploads/big_lots_onboarding/"+os.path.basename(local_path)
    sftp.put(local_path,remote_path)
sftp.close()
transport.close()
print("Done: ",datetime.datetime.now())

Done:  2020-06-17 11:06:27.588641


In [20]:
os.getcwd()

'/home/jian/celery/BiWeekly_New_SignUps'